In [426]:
import pennylane as qml
from pennylane import numpy as np

# 양자 장치 설정
dev = qml.device('default.mixed', wires=2)


@qml.qnode(dev)
def random_circuit(params):
    """임의의 게이트를 포함하는 기본 회로"""

    for i in range(10):
        qml.RX(params[i*3+0], wires=0)
        qml.RY(params[i*3+1], wires=1)
        qml.CNOT(wires=[0, 1])
        qml.RZ(params[i*3+2], wires=0)
    return qml.expval(qml.PauliZ(0)@qml.PauliZ(1))


In [427]:
import pennylane as qml
from pennylane import numpy as np



def extra_polation(circ,theta,noise_factor=[0,1,2,3],p1=0.01,p2=0.02):
    
    real_vaule = circ(theta)
    dev = circ.device
    ops = circ.qtape.operations
    ops_inv = ops[::-1]


    meas = circ.qtape.measurements[0]
    def noise_circ():
        for op in ops:
            eval(f'qml.{op}')
            if len(op.wires)>1:
                for wire in op.wires:
                    qml.DepolarizingChannel(p1, wires=wire)
                    qml.BitFlip(p2, wires=wire)
            else:
                qml.DepolarizingChannel(p1, wires=op.wires) 
                qml.BitFlip(p2, wires=op.wires)
    def noise_circ_inv():
        for op in ops_inv:
            eval(f'qml.adjoint(qml.{op})')
            if len(op.wires)>1:
                for wire in op.wires:
                    qml.DepolarizingChannel(p1, wires=wire)
                    qml.BitFlip(p2, wires=wire)
            else:
                qml.DepolarizingChannel(p1, wires=op.wires) 
                qml.BitFlip(p2, wires=op.wires)
    
    @qml.qnode(dev)
    def extra_polation(factor):
        Z = qml.PauliZ
        Y = qml.PauliY
        X = qml.PauliX
        for i in range(factor):
            noise_circ()
            noise_circ_inv()
        noise_circ()
        return eval(f"qml.{meas}")
    




    res = []
    for factor in noise_factor:
        res.append(extra_polation(factor))
    return res,real_vaule

In [428]:
from torch.utils.data import Dataset, DataLoader
import torch
from tqdm import tqdm
# 데이터셋 생성 클래스
class QuantumData(Dataset):
    def __init__(self, size=500):
        self.data = []
        self.labels = []
        for _ in tqdm(range(size)):
            theta = [np.random.uniform(0, 2 * np.pi) for i in range(30)]
            noise_factor,result = extra_polation(random_circuit,theta,noise_factor=[0,1,2,3],p1=0.01,p2=0.02)
            self.data.append(np.array(noise_factor))
            self.labels.append(result)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return torch.tensor(self.data[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.float32)

# 데이터셋 및 데이터 로더 생성
dataset = QuantumData()
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

  0%|          | 0/500 [00:00<?, ?it/s]

100%|██████████| 500/500 [07:33<00:00,  1.10it/s]


In [429]:
import torch.nn as nn

class RegressionModel(nn.Module):
    def __init__(self):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(4, 50)
        self.fc2 = nn.Linear(50, 20)
        self.fc3 = nn.Linear(20, 1)
        self.relu = nn.GELU()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return torch.squeeze(self.fc3(x))

model = RegressionModel()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
from torch.utils.data import random_split, DataLoader
# 데이터셋 분할
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# 데이터 로더
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# 모델, 손실 함수, 옵티마이저 초기화
model = RegressionModel()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 훈련 및 검증 루프
num_epochs = 5000
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    train_loss /= len(train_loader)
    
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for inputs_val, labels_val in val_loader:
            outputs = model(inputs_val)
            loss = criterion(outputs, labels_val)
            val_loss += loss.item()
    
    val_loss /= len(val_loader)
    
    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

In [456]:
res_MLP = []
res_RAW = []
res_line_list = []

for i in range(36):
    
    print(f'MLP ZEN dist : {abs(model(inputs_val[i]).item()-labels_val[i])}')
    res_MLP.append(abs(model(inputs_val[i]).item()-labels_val[i]))
    print(f'RAW dist : {abs(inputs_val[i][0]-labels_val[i])}')
    res_RAW.append(abs(inputs_val[i][0]-labels_val[i]))
    res_line = np.polyfit([1,3,5,7], inputs_val[i], 1)[1]
    print(f'Line dist : {abs(res_line.item()-labels_val[i])}')
    res_line_list.append(abs(res_line.item()-labels_val[i]))

MLP ZEN dist : 0.015946749597787857
RAW dist : 0.06969409435987473
Line dist : 0.0700213611125946
MLP ZEN dist : 0.01163029670715332
RAW dist : 0.6784425973892212
Line dist : 0.7038582563400269
MLP ZEN dist : 0.07061344385147095
RAW dist : 0.1852310597896576
Line dist : 0.19473418593406677
MLP ZEN dist : 0.11535608768463135
RAW dist : 0.24923928081989288
Line dist : 0.25231462717056274
MLP ZEN dist : 0.036713987588882446
RAW dist : 0.11132106930017471
Line dist : 0.11318791657686234
MLP ZEN dist : 0.0867202952504158
RAW dist : 0.03745127469301224
Line dist : 0.03345983475446701
MLP ZEN dist : 0.16344451904296875
RAW dist : 0.76505446434021
Line dist : 0.7847569584846497
MLP ZEN dist : 0.02339746803045273
RAW dist : 0.08814842253923416
Line dist : 0.0902995690703392
MLP ZEN dist : 0.026457011699676514
RAW dist : 0.39583975076675415
Line dist : 0.4096689522266388
MLP ZEN dist : 0.008375141769647598
RAW dist : 0.027612898498773575
Line dist : 0.029058586806058884
MLP ZEN dist : 0.09321281

In [457]:
np.mean(res_MLP),np.mean(res_RAW),np.mean(res_line_list)

(0.057914753, 0.28760037, 0.29610378)

In [458]:
np.polyfit([1,3,5,7], inputs_val[i], 1)[1]

tensor(0.08350203, requires_grad=True)